# PadPy praca domowa nr 2
## Anna Kozioł

Celem pracy domowej było wykonanie zapytań w języku sql i ich interpretacja oraz wywołania identycznych zapytań przy użyciu pakietu Pandas. 
 Dla każdego z otrzymanych wyników dokonano sprawdzenia, czy  wyniki są tożsame oraz zapytanie 'pandas' jest on klasy DataFrame. Dodatkowo dokonano porówania czasów wykonania poszczególnych obliczeń. Podsumowanie zawiera wnioski z analizy.

### 1. Wczytanie odpowiednich zbiorów danych, metod wyśwetlania oraz utworzenie tymczasowej bazy danych

In [200]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import sqlite3
import tempfile
import os
import timeit

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [201]:
#pliki danych

badges = pd.read_csv("file:///D:/Python/pd2/Badges.txt", sep = ",")
comments = pd.read_csv("file:///D:/Python/pd2/Comments.txt", sep = ",")
postLinks = pd.read_csv("file:///D:/Python/pd2/PostLinks.txt", sep = ",")
posts = pd.read_csv("file:///D:/Python/pd2/Posts.txt", sep = ",")
tags = pd.read_csv("file:///D:/Python/pd2/Tags.txt", sep = ",")
users = pd.read_csv("file:///D:/Python/pd2/Users.txt", sep = ",")
votes = pd.read_csv("file:///D:/Python/pd2/Votes.txt", sep = ",")

baza = os.path.join(tempfile.mkdtemp(), "baza.db")
if os.path.isfile(baza):
    os.remove(baza)
conn = sqlite3.connect(baza)

badges.to_sql("badges", conn)
comments.to_sql("comments", conn)
postLinks.to_sql("postLinks", conn)
posts.to_sql("posts", conn)
tags.to_sql("tags", conn)
users.to_sql("users", conn)
votes.to_sql("votes", conn)


### Zapytanie 1

Zapytanie zwraca dla każdego postu otrzymał ile głosów _'UpMod'_ określone jako UpVotes. 

In [198]:
#sql

start1 = timeit.default_timer()

df_sgl_1 = pd.read_sql_query("SELECT PostId, COUNT(*) AS UpVotes \
                             FROM Votes WHERE VoteTypeId=2 GROUP BY PostId", conn)

t1_sql = timeit.default_timer() - start1

#pandas 

start2 = timeit.default_timer()

x = votes[votes.VoteTypeId == 2][['PostId']].groupby('PostId').size()
df_pd_1 = x.to_frame().reset_index().rename(columns= {0: 'UpVotes'})

t1_pd = timeit.default_timer() - start2

#wynik

display(df_pd_1.head(10))
display(df_sgl_1.head(10))

,PostId,UpVotes
0,1,42
1,2,13
2,3,10
3,4,16
4,5,31
5,6,22
6,7,4
7,8,10
8,9,3
9,10,5


,PostId,UpVotes
0,1,42
1,2,13
2,3,10
3,4,16
4,5,31
5,6,22
6,7,4
7,8,10
8,9,3
9,10,5


* sprawdzenie poprawności

In [92]:
df_sgl_1.equals(df_pd_1)

True

* sprawdzenie typu


In [54]:
type(df_pd_1)

pandas.core.frame.DataFrame

* porównanie czasów

In [199]:
print('sql: ' + str(t1_sql) + 's')
print('pd:  ' + str(t1_pd) + 's')

sql: 0.1934735999966506s
pd:  0.0216186999969068s


### Zapytanie 2 
Zapytanie zwraca Tytuł, Score, licznik wyświetleń (ViewCount),licznik ulubień (FavoriteCount) dla postów,
które są pytaniami oraz mają ponad 10.000 ViewCount i ponad 25 FavoriteCount.

In [202]:
#sql

start1 = timeit.default_timer()

df_sgl_2 = pd.read_sql_query("SELECT Title, Score, ViewCount, FavoriteCount\
                           FROM Posts WHERE PostTypeId=1 AND FavoriteCount >= 25 AND ViewCount>=10000", conn)
t2_sql = timeit.default_timer() - start1

#pandas 
start2 = timeit.default_timer()

df_pd_2 = posts[["Title", "Score", "ViewCount", "FavoriteCount"]][(posts.PostTypeId == 1) & (posts.FavoriteCount >= 25) & (posts.ViewCount>=10000)] \
        .reset_index(drop= True)

t2_pd = timeit.default_timer() - start2
#wyniki

display(df_pd_2.head(10))
display(df_sgl_2.head(10))

,Title,Score,ViewCount,FavoriteCount
0,Terminology index - a list of bike part names ...,165,129577.0,136.0
1,How to clean up at work after a commute?,101,40669.0,25.0
2,What's your most useful cycle accessory?,35,10024.0,30.0
3,Why ride a fixed-gear bike?,171,196543.0,46.0
4,How do I deal with a dog chasing me when I'm t...,109,50035.0,32.0


,Title,Score,ViewCount,FavoriteCount
0,Terminology index - a list of bike part names ...,165,129577.0,136.0
1,How to clean up at work after a commute?,101,40669.0,25.0
2,What's your most useful cycle accessory?,35,10024.0,30.0
3,Why ride a fixed-gear bike?,171,196543.0,46.0
4,How do I deal with a dog chasing me when I'm t...,109,50035.0,32.0


* sprawdzenie poprawności

In [91]:
df_sgl_2.equals(df_pd_2)

True

* sprawdzenie typu

In [53]:
type(df_pd_2)

pandas.core.frame.DataFrame

* porówanie czasów

In [203]:
print('sql: ' + str(t2_sql) + 's')
print('pd:  ' + str(t2_pd) + 's')

sql: 0.01389270000072429s
pd:  0.004494599997997284s


### Zapytanie 3 

Zapytanie zwraca nazwę Tagu, liczbę występowania (Count), Id autora, lokalizację oraz nick (DisplayName) 
dla autorów innych niż o Id = -1 w porządku malejącym względem liczby Count.



In [210]:
#sql

start1 = timeit.default_timer()

df_sgl_3 = pd.read_sql_query("SELECT Tags.TagName, Tags.Count, Posts.OwnerUserId,\
                             Users.Location, Users.DisplayName \
                             FROM Tags \
                             JOIN Posts ON Posts.Id=Tags.WikiPostId \
                             JOIN Users ON Users.AccountId=Posts.OwnerUserId \
                             WHERE OwnerUserId != -1\
                             ORDER BY Count DESC", conn)
t3_sql = timeit.default_timer() - start1

#pandas 
start2 = timeit.default_timer()

temp1 = pd.merge(tags,posts, how = 'inner', left_on = 'WikiPostId', right_on= 'Id' )
temp2=  pd.merge(temp1,users, how = 'inner', left_on = 'OwnerUserId', right_on= 'AccountId' )
df_pd_3 = temp2[temp2.OwnerUserId != -1].sort_values("Count", ascending=[False])[["TagName","Count", "OwnerUserId", "Location", "DisplayName"]]\
        .reset_index(drop = True)
t3_pd = timeit.default_timer() - start2

#wyniki

display(df_pd_3.head(10))
display(df_sgl_3.head(10))

,TagName,Count,OwnerUserId,Location,DisplayName
0,freewheel,167,3021.0,"Moscow, Russia",jfs
1,preventative-maintenance,62,7044.0,"Manchester, United Kingdom",Codebeef
2,7-speed,21,7044.0,"Manchester, United Kingdom",Codebeef
3,lbs,15,3649.0,NaN,Tim
4,reliability,5,3927.0,"Liverpool, United Kingdom",Will Boyce
5,sociable,2,7044.0,"Manchester, United Kingdom",Codebeef
6,ordinary,1,7044.0,"Manchester, United Kingdom",Codebeef
7,dicycle,1,7044.0,"Manchester, United Kingdom",Codebeef


,TagName,Count,OwnerUserId,Location,DisplayName
0,freewheel,167,3021.0,"Moscow, Russia",jfs
1,preventative-maintenance,62,7044.0,"Manchester, United Kingdom",Codebeef
2,7-speed,21,7044.0,"Manchester, United Kingdom",Codebeef
3,lbs,15,3649.0,None,Tim
4,reliability,5,3927.0,"Liverpool, United Kingdom",Will Boyce
5,sociable,2,7044.0,"Manchester, United Kingdom",Codebeef
6,ordinary,1,7044.0,"Manchester, United Kingdom",Codebeef
7,dicycle,1,7044.0,"Manchester, United Kingdom",Codebeef


* sprawdzenie poprawności

In [89]:
df_sgl_3.equals(df_pd_3)

True

* sprawdzenie typu

In [65]:
type(df_pd_3)

pandas.core.frame.DataFrame

* porównanie czasów

In [211]:
print('sql: ' + str(t3_sql) + 's')
print('pd:  ' + str(t3_pd) + 's')

sql: 0.09365570000227308s
pd:  0.025771500004339032s


### Zapytanie 4 

Zapytanie zwraca tytuł postu oraz liczbę powiązanych postów (RelatedPostId) określoną jako NumLinks dla postów które są pytaniem w porządku malejącym względem NumLinks.

In [207]:
#sql
start1 = timeit.default_timer()

df_sgl_4 = pd.read_sql_query("SELECT Posts.Title, RelatedTab.NumLinks FROM \
                            (SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks \
                            FROM PostLinks GROUP BY RelatedPostId) AS RelatedTab \
                            JOIN Posts ON RelatedTab.PostId=Posts.Id \
                            WHERE Posts.PostTypeId=1 \
                            ORDER BY NumLinks DESC", conn)
t4_sql = timeit.default_timer() - start1


#pandas

start2 = timeit.default_timer()

RelatedTab_temp = postLinks.groupby('RelatedPostId').size()
RelatedTab_temp = RelatedTab_temp.to_frame().reset_index()
RelatedTab = RelatedTab_temp.rename(columns = {'RelatedPostId': 'PostId', 0: 'NumLinks'})
posts_temp = posts[posts.PostTypeId == 1]
df_pd_4 = pd.merge(RelatedTab, posts_temp, how = 'inner', right_on ='Id', left_on = 'PostId' ) \
        .sort_values("NumLinks", ascending=[False],kind= 'mergesort')[['Title', 'NumLinks']].reset_index(drop = True)
t4_pd = timeit.default_timer() - start2

#wyniki

display(df_pd_4.head(10))
display(df_sgl_4.head(10))

,Title,NumLinks
0,Terminology index - a list of bike part names ...,156
1,Why shouldn't I care what model/make/year my b...,154
2,How can I tell what year my bike was made?,100
3,What is the maximum or minimum tire width I ca...,42
4,Front suspension or full suspension mountain bike,37
5,What are the differences (and their practical ...,33
6,Picking a bike for a cyclist new to riding on ...,33
7,Bike recommendation for daily commute,32
8,Advice on buying a new bike,32
9,What makes a good entry level road bike?,30


,Title,NumLinks
0,Terminology index - a list of bike part names ...,156
1,Why shouldn't I care what model/make/year my b...,154
2,How can I tell what year my bike was made?,100
3,What is the maximum or minimum tire width I ca...,42
4,Front suspension or full suspension mountain bike,37
5,What are the differences (and their practical ...,33
6,Picking a bike for a cyclist new to riding on ...,33
7,Bike recommendation for daily commute,32
8,Advice on buying a new bike,32
9,What makes a good entry level road bike?,30


* poprawność wyników

In [88]:
df_sgl_5.equals(df_pd_5)

True

* sprawdzenie typu

In [51]:
type(df_pd_4)

pandas.core.frame.DataFrame

* porównanie czasów

In [208]:
print('sql: ' + str(t4_sql) + 's')
print('pd:  ' + str(t4_pd) + 's')

sql: 0.03690020000067307s
pd:  0.03320499999972526s


### Zapytanie 5 

Zapytanie zwraca Id Postu liczbę głosów 'za' _(UpMod)_ określonych jako _'UpVotes'_ dla postów będących pytaniem w porządku malejącym względem UpVotes. 

In [212]:
#sql
start1 = timeit.default_timer()
df_sgl_5 = pd.read_sql_query("SELECT UpVotesTab.*, Posts.Title FROM \
                            ( \
                            SELECT PostId, COUNT(*) AS UpVotes FROM Votes WHERE VoteTypeId=2 GROUP BY PostId \
                            ) AS UpVotesTab \
                            JOIN Posts ON UpVotesTab.PostId=Posts.Id \
                            WHERE Posts.PostTypeId=1 \
                            ORDER BY UpVotesTab.UpVotes DESC", conn)
t5_sql = timeit.default_timer() - start1

#pandas

start2 = timeit.default_timer()

UpVotesTab = votes[votes.VoteTypeId == 2 ].groupby('PostId').size()
UpVotesTab = UpVotesTab.to_frame().reset_index().rename(columns = { 0: 'UpVotes'})
posts_temp = posts[posts.PostTypeId == 1][['Title', 'Id']]
df_pd_5 = pd.merge(UpVotesTab, posts_temp, right_on = 'Id', left_on = 'PostId', how = 'inner' ) \
         .sort_values('UpVotes', ascending=False,kind= 'mergesort').drop('Id', axis='columns').reset_index(drop = True)

t5_pd = timeit.default_timer() - start2
#wyniki

display(df_pd_5.head(10))
display(df_sgl_5.head(10))


,PostId,UpVotes,Title
0,1980,174,Why ride a fixed-gear bike?
1,244,167,Terminology index - a list of bike part names ...
2,8233,110,How do I deal with a dog chasing me when I'm t...
3,308,103,How to clean up at work after a commute?
4,2294,94,"Safety data: Which is safer, head/tail lights ..."
5,1500,92,Why are disc brakes rare on road bikes?
6,2446,87,Why do cyclists prefer messenger bags over bac...
7,2170,79,Is pollution a problem in urban cycling?
8,57082,79,How to pass a bicycle while driving a car
9,14372,77,Can I use WD-40 on my bike?


,PostId,UpVotes,Title
0,1980,174,Why ride a fixed-gear bike?
1,244,167,Terminology index - a list of bike part names ...
2,8233,110,How do I deal with a dog chasing me when I'm t...
3,308,103,How to clean up at work after a commute?
4,2294,94,"Safety data: Which is safer, head/tail lights ..."
5,1500,92,Why are disc brakes rare on road bikes?
6,2446,87,Why do cyclists prefer messenger bags over bac...
7,2170,79,Is pollution a problem in urban cycling?
8,57082,79,How to pass a bicycle while driving a car
9,14372,77,Can I use WD-40 on my bike?


* porównanie wyników 

In [84]:
df_sgl_5.equals(df_pd_5)

True

* sprawdzenie typu

In [85]:
type(df_pd_5)

pandas.core.frame.DataFrame

* porównanie czasów 

In [214]:
print('sql: ' + str(t5_sql) + 's')
print('pd:  ' + str(t5_pd) + 's')

sql: 0.2556944000025396s
pd:  0.04737699999532197s


### Zapytanie 6 

Zapytanie dla każdego Id postu który otrzymał głos 'za' ('UpMod') zwraca liczbę tych głosów oraz liczbę głosów 'przeciw' (DownMod). Gdy nie ma informacji o głosach 'DownMod', zwracane jest 0.


In [229]:
#sql

start1 = timeit.default_timer()
df_sgl_6 = pd.read_sql_query("SELECT UpVotesTab.PostId, UpVotesTab.UpVotes, IFNULL(DownVotesTab.DownVotes, 0) AS DownVotes \
                            FROM \
                            (\
                            SELECT PostId, COUNT(*) AS UpVotes FROM Votes\
                            WHERE VoteTypeId=2 GROUP BY PostId\
                            ) AS UpVotesTab\
                            LEFT JOIN\
                            (\
                            SELECT PostId, COUNT(*) AS DownVotes FROM Votes\
                            WHERE VoteTypeId=3 GROUP BY PostId\
                            ) AS DownVotesTab\
                            ON UpVotesTab.PostId=DownVotesTab.PostId", conn)
                            
t6_sql = timeit.default_timer() - start1

#pandas

start2 = timeit.default_timer()

UpVotesTab_temp = votes[votes.VoteTypeId == 2].groupby('PostId').size()
UpVotesTab = UpVotesTab_temp.to_frame().reset_index().rename(columns = { 0: 'UpVotes'})

DownVotesTab_temp = votes[votes.VoteTypeId == 3].groupby('PostId').size()
DownVotesTab = DownVotesTab_temp.to_frame().reset_index().rename(columns = { 0: 'DownVotes'})

df_pd_6 = pd.merge(UpVotesTab,DownVotesTab, how = 'left', on = 'PostId')
df_pd_6['DownVotes'] = df_pd_6['DownVotes'].replace(np.nan, 0)
df_pd_6['DownVotes'] = df_pd_6['DownVotes'].astype(np.int64)

t6_pd = timeit.default_timer() - start2

# wyniki 

display(df_pd_6.head(10))
display(df_sgl_6.head(10))


,PostId,UpVotes,DownVotes
0,1,42,0
1,2,13,0
2,3,10,0
3,4,16,0
4,5,31,0
5,6,22,0
6,7,4,0
7,8,10,0
8,9,3,0
9,10,5,1


,PostId,UpVotes,DownVotes
0,1,42,0
1,2,13,0
2,3,10,0
3,4,16,0
4,5,31,0
5,6,22,0
6,7,4,0
7,8,10,0
8,9,3,0
9,10,5,1


* porównanie wyników

In [224]:

df_sgl_6.equals(df_pd_6) 


True

* sprawdzenie typu

In [182]:
type(df_pd_6)

pandas.core.frame.DataFrame

* porównanie czasów

In [225]:
print('sql: ' + str(t6_sql) + 's')
print('pd:  ' + str(t6_pd) + 's')

sql: 0.30428329999995185s
pd:  0.036605300003429875s


### Zapytanie 7

Zapytanie zwraca Id postu oraz różnicę w ilości głosów 'za' (UpMod) i 'przeciw' (DownMod).


In [226]:
#sql

start1 = timeit.default_timer()
df_sgl_7 = pd.read_sql_query("SELECT PostId, UpVotes-DownVotes AS Votes FROM ( \
                            SELECT UpVotesTab.PostId, UpVotesTab.UpVotes, IFNULL(DownVotesTab.DownVotes, 0) AS DownVotes\
                            FROM\
                            (\
                            SELECT PostId, COUNT(*) AS UpVotes FROM Votes\
                            WHERE VoteTypeId=2 GROUP BY PostId\
                            ) AS UpVotesTab\
                            LEFT JOIN\
                            (\
                            SELECT PostId, COUNT(*) AS DownVotes\
                            FROM Votes WHERE VoteTypeId=3 GROUP BY PostId\
                            ) AS DownVotesTab\
                            ON UpVotesTab.PostId=DownVotesTab.PostId\
                            UNION\
                            SELECT DownVotesTab.PostId, IFNULL(UpVotesTab.UpVotes, 0) AS UpVotes, DownVotesTab.DownVotes\
                            FROM\
                            (\
                            SELECT PostId, COUNT(*) AS DownVotes FROM Votes\
                            WHERE VoteTypeId=3 GROUP BY PostId\
                            ) AS DownVotesTab\
                            LEFT JOIN\
                            (\
                            SELECT PostId, COUNT(*) AS UpVotes FROM Votes\
                            WHERE VoteTypeId=2 GROUP BY PostId\
                            ) AS UpVotesTab\
                            ON DownVotesTab.PostId=UpVotesTab.PostId\
                            )", conn)
t7_sql = timeit.default_timer() - start1
                            
#pandas

start2 = timeit.default_timer()

UpVotesTab_temp = votes[votes.VoteTypeId == 2].groupby('PostId').size()
UpVotesTab = UpVotesTab_temp.to_frame().reset_index().rename(columns = { 0: 'UpVotes'}) 
DownVotesTab_temp = votes[votes.VoteTypeId == 3].groupby('PostId').size()
DownVotesTab = DownVotesTab_temp.to_frame().reset_index().rename(columns = { 0: 'DownVotes'}) 

left = pd.merge(UpVotesTab,DownVotesTab, how = 'left', on = 'PostId' )
left2 = pd.merge(DownVotesTab,UpVotesTab, how = 'left', on = 'PostId' )
left2['UpVotes'] = left2['UpVotes'].replace(np.nan, 0)
left2['UpVotes'] = left2['UpVotes'].astype(int)


df_pd_7 = pd.concat([left,left2],sort=False).drop_duplicates()
df_pd_7['DownVotes'] = df_pd_7['DownVotes'].replace(np.nan, 0)
df_pd_7['DownVotes'] = df_pd_7['DownVotes'].astype(int)
df_pd_7['Votes'] = df_pd_7['UpVotes'] - df_pd_7['DownVotes']
df_pd_7 = df_pd_7[['PostId','Votes']].sort_values('PostId', ascending=True, kind= 'mergesort').reset_index(drop = True)

t7_pd = timeit.default_timer() - start2

#wyniki

display(df_pd_7.head(11))
display(df_sgl_7.head(11))

,PostId,Votes
0,1,42
1,2,13
2,3,10
3,4,16
4,5,31
5,6,22
6,7,4
7,8,10
8,9,3
9,10,4


,PostId,Votes
0,1,42
1,2,13
2,3,10
3,4,16
4,5,31
5,6,22
6,7,4
7,8,10
8,9,3
9,10,4


* porówanie wyników

In [179]:
df_sgl_7.equals(df_pd_7)

True

* sprawdzenie typu 

In [138]:
type(df_pd_7)

pandas.core.frame.DataFrame

* porównanie czasów

In [227]:
print('sql: ' + str(t7_sql) + 's')
print('pd:  ' + str(t7_pd) + 's')

sql: 0.5894488000049023s
pd:  0.10622239999793237s


### Podsumowanie 
We wszystkich przypadkach udało się uzyskać te same wyniki przy pomocy pd.read_sql_query oraz funkcji pakietu pandas.
Wszystkie zwracane wyniki są klasy DataFrame.
Porównanie czasów prowadzi do wniosku, iż zazwyczaj funkcje pakietu 'Pandas' są wiele szybsze, w szczególności w przypadku bardzo złożonych zapytań.
Tylko w przypadku 4 funkcja read_sql_query okazała się minimalnie szybsza. 